<a href="https://colab.research.google.com/github/roklp/MLP34/blob/main/kaggle3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/kaggle1/train.csv', index_col=0)

df.head()

In [16]:
columns_list = df.columns.tolist()
print(columns_list)

['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad']


In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

scaler = StandardScaler()

df_numerical = df.select_dtypes(include=['float64', 'int64'])  # 숫자형 열만 추출

scaler.fit(df_numerical)
numerical_scaled = scaler.transform(df_numerical)
df_numerical_scaled = pd.DataFrame(data=numerical_scaled, columns=df_numerical.columns)

# 스케일링된 데이터의 통계량 출력
print('feature들의 평균 값')
print(df_numerical_scaled.mean())
print('\nfeature들의 분산 값')
print(df_numerical_scaled.var())


[1 1 1 ... 0 1 1]


In [13]:
from sklearn.preprocessing import StandardScaler

def apply_preprocessing(data, scaler):
    features = data.copy(deep=True)
    # numerical dataframe
    mask_numeric = features.dtypes == float
    df_numerical = features.loc[:, mask_numeric]
    # categorical dataframe
    mask_categorical = features.dtypes != float
    df_categorical = features.loc[:, mask_categorical]
    # 스케일러를 사용하여 데이터 표준화
    numerical_scaled = scaler.transform(df_numerical)
    df_numerical_scaled = pd.DataFrame(data=numerical_scaled, columns=df_numerical.columns)
    # label encoding
    df_encoded = df_categorical.copy(deep=True)
    df_encoded['Gender'] = df_categorical['Gender'].map({'Male':0, 'Female':1})
    df_encoded['family_history_with_overweight'] = df_categorical['family_history_with_overweight'].map({'no':0, 'yes':1})
    df_encoded['FAVC'] = df_categorical['FAVC'].map({'no':0, 'yes':1})
    df_encoded['CAEC'] = df_categorical['CAEC'].map({'no':0, 'Sometimes':1, 'Frequently':2, 'Always':3})
    df_encoded['SMOKE'] = df_categorical['SMOKE'].map({'no':0, 'yes':1})
    df_encoded['SCC'] = df_categorical['SCC'].map({'no':0, 'yes':1})
    df_encoded['CALC'] = df_categorical['CALC'].map({'no':0, 'Sometimes':1, 'Frequently':2, 'Always':3})
    # one-hot encoding
    df_onehot = pd.get_dummies(df_categorical['MTRANS']).astype(int)
    df_encoded.drop('MTRANS', axis=1, inplace=True)
    # one feature of df_encoded is redundant; we can remove it
    df_encoded = pd.concat([df_encoded, df_onehot.iloc[:, :-1]], axis = 1)
    df_numerical_scaled.reset_index(drop=True, inplace=True)
    df_encoded.reset_index(drop=True, inplace=True)
    # concatenate
    df_all_features = pd.concat([df_numerical_scaled, df_encoded], axis = 1)
    return df_all_features


In [28]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/data/kaggle1/train.csv', index_col=0)

# 전처리 함수 정의
def apply_preprocessing(data, scaler):
    features = data.copy(deep=True)
    # numerical dataframe
    mask_numeric = features.dtypes == float
    df_numerical = features.loc[:, mask_numeric]
    # categorical dataframe
    mask_categorical = features.dtypes != float
    df_categorical = features.loc[:, mask_categorical]
    # 스케일러를 사용하여 데이터 표준화
    numerical_scaled = scaler.transform(df_numerical)
    df_numerical_scaled = pd.DataFrame(data=numerical_scaled, columns=df_numerical.columns)
    # label encoding
    df_encoded = df_categorical.copy(deep=True)
    df_encoded['Gender'] = df_categorical['Gender'].map({'Male':0, 'Female':1})
    df_encoded['FAVC'] = df_categorical['FAVC'].map({'no':0, 'yes':1})
    df_encoded['CAEC'] = df_categorical['CAEC'].map({'no':0, 'Sometimes':1, 'Frequently':2, 'Always':3})
    df_encoded['SMOKE'] = df_categorical['SMOKE'].map({'no':0, 'yes':1})
    df_encoded['SCC'] = df_categorical['SCC'].map({'no':0, 'yes':1})
    df_encoded['CALC'] = df_categorical['CALC'].map({'no':0, 'Sometimes':1, 'Frequently':2, 'Always':3})
    # one-hot encoding
    df_onehot = pd.get_dummies(df_categorical['MTRANS']).astype(int)
    df_encoded.drop('MTRANS', axis=1, inplace=True)
    # one feature of df_encoded is redundant; we can remove it
    df_encoded = pd.concat([df_encoded, df_onehot.iloc[:, :-1]], axis = 1)
    df_numerical_scaled.reset_index(drop=True, inplace=True)
    df_encoded.reset_index(drop=True, inplace=True)
    # concatenate
    df_all_features = pd.concat([df_numerical_scaled, df_encoded], axis = 1)
    return df_all_features

# StandardScaler 객체 생성
scaler = StandardScaler()
# 수치형 열 추출 및 스케일링
df_numerical = df.select_dtypes(include=['float64', 'int64'])
scaler.fit(df_numerical)

# 전처리 수행
df_preprocessed = apply_preprocessing(df, scaler)

# 독립변수와 종속변수 분리
X = df_preprocessed.drop(['family_history_with_overweight', 'NObeyesdad'], axis=1)  # 종속변수와 불필요한 열 제거
y = df_preprocessed['family_history_with_overweight']

# 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

# 종속 변수에 Label Encoding 적용
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# XGBoost 모델 학습
model = xgb.XGBClassifier()
model.fit(X_train, y_train_encoded)

# 예측
y_pred_encoded = model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
print("정확도:", accuracy)


정확도: 0.8622350674373795


In [ ]:
from sklearn.model_selection import GridSearchCV

# XGBoost 모델 생성
model = xgb.XGBClassifier()

# 튜닝할 하이퍼파라미터 범위 지정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3]
}

# GridSearchCV 객체 생성
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)

# 모델 학습 및 튜닝
grid_search.fit(X_train, y_train_encoded)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", grid_search.best_params_)

# 튜닝된 모델로 예측
y_pred_tuned = grid_search.predict(X_test)

# 정확도 평가
accuracy_tuned = accuracy_score(y_test_encoded, y_pred_tuned)
print("조정된 모델의 정확도:", accuracy_tuned)


In [31]:
# 최적의 하이퍼파라미터를 적용하여 XGBoost 모델 생성
best_model = xgb.XGBClassifier(learning_rate=0.01, max_depth=7, n_estimators=300)

# 훈련 데이터로 모델 학습
best_model.fit(X_train, y_train_encoded)

# 테스트 데이터로 예측
y_pred_best = best_model.predict(X_test)

# 정확도 평가
accuracy_best = accuracy_score(y_test_encoded, y_pred_best)
print("최적 모델의 정확도:", accuracy_best)


최적 모델의 정확도: 0.86873795761079


In [33]:
from sklearn.model_selection import GridSearchCV

# 튜닝할 하이퍼파라미터 그리드 생성
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300]
}

# GridSearchCV를 사용하여 모델 튜닝
grid_search = GridSearchCV(estimator=xgb.XGBClassifier(), param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train_encoded)

# 최적의 하이퍼파라미터와 정확도 출력
print("최적의 하이퍼파라미터:", grid_search.best_params_)
print("최적 모델의 정확도:", grid_search.best_score_)


최적의 하이퍼파라미터: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 300}
최적 모델의 정확도: 0.882994275350589


In [34]:
best_model = xgb.XGBClassifier(learning_rate=0.05, max_depth=5, n_estimators=300)

best_model.fit(X_train, y_train_encoded)

y_pred_best = best_model.predict(X_test)

accuracy_best = accuracy_score(y_test_encoded, y_pred_best)
print("최적 모델의 정확도:", accuracy_best)


최적 모델의 정확도: 0.8709055876685935


In [35]:
from sklearn.model_selection import cross_val_score

best_model = xgb.XGBClassifier(learning_rate=0.05, max_depth=5, n_estimators=300)

# 교차 검증 수행
cv_scores = cross_val_score(best_model, X_train, y_train_encoded, cv=5)  # 5-fold cross-validation

# 교차 검증 결과 출력
print("교차 검증 정확도:", cv_scores)
print("평균 정확도:", cv_scores.mean())


교차 검증 정확도: [0.87537628 0.88527552 0.88286661 0.88256549 0.88467329]
평균 정확도: 0.8821514377428876


In [36]:
from sklearn.model_selection import GridSearchCV

# XGBoost 모델 생성
model = xgb.XGBClassifier()

# 하이퍼파라미터 그리드 설정
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300]
}

# 그리드 서치 수행
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train_encoded)

# 최적의 하이퍼파라미터와 정확도 출력
print("최적의 하이퍼파라미터:", grid_search.best_params_)
print("최적 모델의 정확도:", grid_search.best_score_)


최적의 하이퍼파라미터: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 200}
최적 모델의 정확도: 0.8829342075613547


In [37]:
best_model = xgb.XGBClassifier(learning_rate=0.05, max_depth=5, n_estimators=200)

best_model.fit(X_train, y_train_encoded)

y_pred_best = best_model.predict(X_test)

accuracy_best = accuracy_score(y_test_encoded, y_pred_best)
print("최적 모델의 정확도:", accuracy_best)


최적 모델의 정확도: 0.8704238921001927


In [40]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
import xgboost as xgb
from imblearn.over_sampling import SMOTE

# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/data/kaggle1/train.csv', index_col=0)

# 전처리 함수 정의
def apply_preprocessing(data, scaler):
    features = data.copy(deep=True)
    mask_numeric = features.dtypes == float
    df_numerical = features.loc[:, mask_numeric]
    mask_categorical = features.dtypes != float
    df_categorical = features.loc[:, mask_categorical]
    numerical_scaled = scaler.transform(df_numerical)
    df_numerical_scaled = pd.DataFrame(data=numerical_scaled, columns=df_numerical.columns)
    df_encoded = df_categorical.copy(deep=True)
    df_encoded['Gender'] = df_categorical['Gender'].map({'Male':0, 'Female':1})
    df_encoded['FAVC'] = df_categorical['FAVC'].map({'no':0, 'yes':1})
    df_encoded['CAEC'] = df_categorical['CAEC'].map({'no':0, 'Sometimes':1, 'Frequently':2, 'Always':3})
    df_encoded['SMOKE'] = df_categorical['SMOKE'].map({'no':0, 'yes':1})
    df_encoded['SCC'] = df_categorical['SCC'].map({'no':0, 'yes':1})
    df_encoded['CALC'] = df_categorical['CALC'].map({'no':0, 'Sometimes':1, 'Frequently':2, 'Always':3})
    df_onehot = pd.get_dummies(df_categorical['MTRANS']).astype(int)
    df_encoded.drop('MTRANS', axis=1, inplace=True)
    df_encoded = pd.concat([df_encoded, df_onehot.iloc[:, :-1]], axis = 1)
    df_numerical_scaled.reset_index(drop=True, inplace=True)
    df_encoded.reset_index(drop=True, inplace=True)
    df_all_features = pd.concat([df_numerical_scaled, df_encoded], axis = 1)
    return df_all_features

# StandardScaler 객체 생성
scaler = StandardScaler()
df_numerical = df.select_dtypes(include=['float64', 'int64'])
scaler.fit(df_numerical)

# 전처리 수행
df_preprocessed = apply_preprocessing(df, scaler)

# 독립변수와 종속변수 분리
X = df_preprocessed.drop(['family_history_with_overweight', 'NObeyesdad'], axis=1)
y = df_preprocessed['family_history_with_overweight']

# 종속 변수 Label Encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# SMOTE 적용
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y_encoded)

# 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# XGBoost 모델 생성
model = xgb.XGBClassifier()

# 교차 검증 수행
cv_scores = cross_val_score(model, X_train, y_train, cv=5)
print("교차 검증 정확도:", cv_scores)
print("평균 정확도:", cv_scores.mean())

# 하이퍼파라미터 튜닝
best_model = xgb.XGBClassifier(learning_rate=0.05, max_depth=5, n_estimators=200)
best_model.fit(X_train, y_train)
y_pred_best = best_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
print("최적 모델의 정확도:", accuracy_best)


교차 검증 정확도: [0.90707071 0.90174472 0.89952241 0.89713446 0.90668626]
평균 정확도: 0.9024317114097358
최적 모델의 정확도: 0.8851013811342933
